In [2]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [5]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [6]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [7]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [8]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [9]:
#code adapted from lecture 18

glove_wiki_vectors.most_similar('basketball')

[('football', 0.855563759803772),
 ('hockey', 0.8041675090789795),
 ('soccer', 0.8003972172737122),
 ('nba', 0.7771991491317749),
 ('baseball', 0.7645360827445984),
 ('volleyball', 0.7434347867965698),
 ('softball', 0.7363322377204895),
 ('sports', 0.7128217816352844),
 ('team', 0.7124603390693665),
 ('ncaa', 0.7113980650901794)]

In [10]:
glove_wiki_vectors.most_similar('lebron')

[('dwyane', 0.7860801219940186),
 ('shaquille', 0.7481709718704224),
 ('bosh', 0.7158374190330505),
 ("o'neal", 0.6786481738090515),
 ('carmelo', 0.6780193448066711),
 ('shaq', 0.6595335602760315),
 ('iverson', 0.6399611830711365),
 ('cavaliers', 0.6334389448165894),
 ('bryant', 0.63310706615448),
 ('olajuwon', 0.6275714635848999)]

In [11]:
glove_wiki_vectors.most_similar('cheesecake')

[('chocolate', 0.7293441295623779),
 ('brownies', 0.6957851648330688),
 ('custard', 0.6954904198646545),
 ('caramel', 0.6932608485221863),
 ('cookies', 0.6721082329750061),
 ('cake', 0.6698510646820068),
 ('pie', 0.6569927930831909),
 ('pecan', 0.6563494205474854),
 ('dessert', 0.6547815799713135),
 ('cream', 0.6479873657226562)]

In [12]:
glove_wiki_vectors.most_similar('tiramisu')

[('flourless', 0.645910382270813),
 ('brulee', 0.64039546251297),
 ('applesauce', 0.6240904927253723),
 ('mousse', 0.6139968633651733),
 ('gelato', 0.6055755019187927),
 ('dessert', 0.6023645997047424),
 ('aperitif', 0.6001883745193481),
 ('buttercream', 0.592993974685669),
 ('torte', 0.5871951580047607),
 ('flan', 0.5761141777038574)]

In [13]:
glove_wiki_vectors.most_similar('burma')

[('myanmar', 0.8043321371078491),
 ('laos', 0.7390782237052917),
 ('cambodia', 0.7248762249946594),
 ('burmese', 0.7016641497612),
 ('rangoon', 0.6855918765068054),
 ('nepal', 0.6512065529823303),
 ('bhutan', 0.6434482336044312),
 ('vietnam', 0.6383993625640869),
 ('thailand', 0.6122615933418274),
 ('junta', 0.5998669862747192)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [15]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [16]:
#code adapted from lecture 18

for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], glove_wiki_vectors.similarity(pair[0], pair[1]))
    )

The similarity between coast and shore is 0.700
The similarity between clothes and closet is 0.546
The similarity between old and new is 0.643
The similarity between smart and intelligent is 0.755
The similarity between dog and cat is 0.880
The similarity between tree and lawyer is 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.  

In [17]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [18]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [19]:
glove_wiki_vectors.similarity("white", "rich")

0.4472359

In [20]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

In [21]:
#1
glove_wiki_vectors.similarity("man", "construction")

0.29262656

In [22]:
glove_wiki_vectors.similarity("woman", "construction")

0.2318834

In [23]:
#2
glove_wiki_vectors.similarity("chef", "man")

0.3336456

In [24]:

glove_wiki_vectors.similarity("chef", "woman")

0.3215433

In [25]:
#3
analogy("man", "education", "woman")

man : education :: woman : ?


,Analogy word,Score
0,vocational,0.702333
1,educational,0.700390
2,schools,0.679494
3,health,0.673882
4,schooling,0.671382
5,curriculum,0.658734
6,nursing,0.657793
7,academic,0.653934
8,literacy,0.645650
9,teaching,0.645260


In [78]:
#4
analogy("hero", "good", "villain")

hero : good :: villain : ?


,Analogy word,Score
0,bad,0.687884
1,thing,0.679428
2,better,0.673386
3,things,0.671866
4,'re,0.652626
5,kind,0.648516
6,enough,0.639457
7,sort,0.638219
8,terrific,0.637727
9,really,0.635683


1. A slight bias is noted towards the word "men" and "construction". This isn't too surprising to see as blue-collar work has long been associated with males.
2. There is also a slight bias towards "man" and "chef" in comparison to "woman" and "chef", which plays into the stereotype that men are usually chefs.
3. The word vector associates women with vocational education the most and shows bias towards woman being in fields such as nursing and teaching.
4. Villains are associated with "bad" the most; however, the other analogy words such as "kind" and "terrific" are are not as predictable because they don't play into teh stereotype of what a villain is.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. Based on the exploration, the quality of the pre-trained embeddings still tend to have biases even though most of them have been de-biased. It is hard to quantitatively judge the quality of embeddings, however it doesn't seem like any of the results show tremendous bias in any one direction.
2. Using a model trained on biased embeddings could result in a model/algorithm that recommends one thing over another solely based on a gender input. It could predict things that are not necessarily right/true but are just predicted solely because that's what the model has seen. Certain genders/races/sexuality could be discriminated against because of machine learned outputs.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Classification with pre-trained embeddings 
rubric={points:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`) and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. 
2. Train logistic regression with average embedding representation extracted using spaCy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [38]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [39]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [40]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [45]:
import spacy

nlp = spacy.load("en_core_web_md")

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Type your answer here, replacing this text._

In [43]:
#1
# code adapted from lecture 10
from sklearn.metrics import classification_report

pipe = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000)
)
pipe.fit(X_train,y_train)
bow_report=print(classification_report(
        y_test, pipe.predict(X_test)))
bow_report

                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [46]:
#2
# code adapted from lecture 18
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

In [47]:
lgr = LogisticRegression(max_iter=10000)
lgr.fit(X_train_embeddings, y_train)

LogisticRegression(max_iter=10000)

In [48]:
embed_report=print(classification_report(
        y_test, lgr.predict(X_test_embeddings)))
embed_report


                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.57      0.51      0.54       469
        exercise       0.68      0.76      0.72        74
         leisure       0.72      0.65      0.68       407
          nature       0.69      0.72      0.70        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.74      4238
    weighted avg       0.79      0.79      0.79      4238



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. CountVectorizer takes out context and relationships between words and only produces counts of words within documents while the average embedding approach obtains representations of documents by obtaining the average word embedding (vector) across the entire document. Embeddings provide more context by taking into account word proximity in a vector space.
2. CountVectorizer (sparse representation) is easier to interpret because it classifies texts based on the counts of unique word occurrences within the different documents. Word embeddings (dense representation) are less interpretable because the features within each word vector are hard to describe as they have undergone some type of "transformation".
3. Transfer learning is employed since the pre-trained model "en_core_web_md" is used as a starting point to obtain word embeddings. The benefits reaped from using this model is that the word embedding outputs obtained with the "en_core_web_md" model are repurposed/inputted into a logistic regression model to help us optimize and improve the predictions for the regression model.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [139]:
from sklearn.datasets import fetch_20newsgroups

In [140]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [141]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Type your answer here, replacing this text._

In [142]:
import nltk
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/madihung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [143]:
# From lecture 18
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE", "NUM", "SYM"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [144]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [145]:
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. prov...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bug appreciate help create w...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify subject patch x11r5 r...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference feds mandate l...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure x11r5 need c...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

We used a preprocess function to:
1. removing stopwords - most frequent occuring words in english are not very useful
2. removing words with less than two letters - these words will probably not provide insight into the topics of documents
3. removing irrelevant characters and words such as punctuation, contractions, adverbs, pronouns, and spaces - these are usually noise and do not give any context to what topic is being discussed
4. lemmatization - to help group words that have the same meaning together

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

In [146]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [147]:
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

# Code adapted from lecture 18
n_topics = 3
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)
document_topics = lda.fit_transform(X)

In [148]:
print("lda.components_.shape: {}".format(lda.components_.shape))

lda.components_.shape: (3, 36189)


In [149]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [150]:
import mglearn

mglearn.tools.print_topics(
    topics=range(3),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       
--------      --------      --------      
people        file          game          
god           program       team          
know          use           year          
think         image         play          
say           window        good          
gun           edu           say           
time          available     think         
believe       include       player        
come          run           win           
thing         server        know          




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [156]:
# Code adapted from lecture 18
import gensim
import gensim.corpora as corpora

corpus = [doc.split() for doc in df["text_pp"].tolist()]
dictionary = corpora.Dictionary(corpus)  # Create a vocabulary for the lda model
pd.DataFrame(
    dictionary.token2id.keys(), index=dictionary.token2id.values(), columns=["Word"]
)

,Word
0,artificial
1,claim
2,constitional
3,constitution
4,court
...,...
37006,lezgians
37007,routed
37008,sevan
37009,territorium


In [152]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]
doc_term_matrix[1][:20]

[(10, 2),
 (24, 3),
 (41, 3),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 6),
 (47, 2),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 2),
 (54, 2),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1)]

In [157]:
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.models import LdaModel

num_topics = 3

lda = gensim.models.LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    passes=10,
)

In [158]:
lda.print_topics(num_words=4)  # Topics

[(0, '0.007*"team" + 0.007*"game" + 0.006*"year" + 0.005*"play"'),
 (1, '0.009*"god" + 0.009*"people" + 0.008*"think" + 0.007*"know"'),
 (2, '0.011*"file" + 0.006*"program" + 0.006*"image" + 0.006*"use"')]

In [159]:
# Document 1
print("Document: ", df.iloc[0][0])
print("Topic assignment for document: ", lda[doc_term_matrix[0]])

Document:  You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by the Supreme Court that they didn't have that right?



Topic assignment for document:

In [161]:
# Document 2
print("Document: ", df.iloc[1][0])
print("Topic assignment for document: ", lda[doc_term_matrix[1]])

Document:  


It's not a bad question: I don't have any refs that list this algorithm
either. But thinking about it a bit, it shouldn't be too hard.

1) Take three of the points and find the plane they define as well as
the circle that they lie on (you say you have this algorithm already)

2) Find the center  of this circle. The line passing through this center
perpendicular to the plane of the three points passes through the center of
the sphere.

3) Repeat with the unused point and two of the original points. This
gives you two different lines that both pass through the sphere's
origin. Their interection is the center of the sphere.

4) the radius is easy to compute, it's just the distance from the center to
any of the original points.

I'll leave the math to you, but this is a workable algorithm. :-)


An alternate method would be to take pairs of points: the plane formed
by the perpendicular bisector of each line segment pair also contains the
center of the sphere. Three pairs will

In [162]:
# Document 3
print("Document: ", df.iloc[2][0])
print("Topic assignment for document: ", lda[doc_term_matrix[2]])

Document:  
Actuallay I don't, but on the other hand I don't support the idea of having
one newsgroup for every aspect of graphics programming as proposed by Brian,
in his reply to my original posting.
I would suggest a looser structure more like a comp.graphics.programmer,
comp.graphics.hw_and_sw
The reason for making as few groups as possible is for the same reason you
say we shouldn't spilt up, not to get to few postings every day.
I takes to much time to browse through all postings just to find two or 
three I'm interested in.

I understand and agree when you say you want all aspects of graphics in one
meeting. I agree to some extension. I see news as a forum to exchange ideas,
help others or to be helped. I think this is difficult to achive if there
are so many different things in one meeting.

Good evening netters|-)
Topic assignment for document:  [(1, 0.7550497), (2, 0.2384003)]


In [163]:
# Document 4
print("Document: ", df.iloc[3][0])
print("Topic assignment for document: ", lda[doc_term_matrix[3]])

Document:  The following problem is really bugging me,
and I would appreciate any help.

I create two windows:

w1 (child to root) with event_mask = ButtonPressMask|KeyPressMask;
w2 (child to w1) with do_not_propagate_mask = ButtonPressMask|KeyPressMask;


Keypress events in w2 are discarded, but ButtonPress events fall through
to w1, with subwindow set to w2.

FYI, I'm using xnews/olvwm.

Am I doing something fundamentally wrong here?
Topic assignment for document:  [(0, 0.013081095), (1, 0.53478134), (2, 0.4521376)]


In [164]:
# Document 5
print("Document: ", df.iloc[4][0])
print("Topic assignment for document: ", lda[doc_term_matrix[4]])

Document:  

  This is the latest from UPI 

     Foreign Ministry spokesman Ferhat Ataman told journalists Turkey was
     closing its air space to all flights to and from Armenia and would
     prevent humanitarian aid from reaching the republic overland across
     Turkish territory.

  
   Historically even the most uncivilized of peoples have exhibited 
   signs of compassion by allowing humanitarian aid to reach civilian
   populations. Even the Nazis did this much.

   It seems as though from now on Turkey will publicly pronounce 
   themselves 'hypocrites' should they choose to continue their
   condemnation of the Serbians.



--
Topic assignment for document:  [(0, 0.9819853)]


1. Document 1 is about gun laws and was assigned to Topic 1. This was unexpected because it seems that a document on guns would be put into Topic 0. 
2. Document 2 si about a math algorithm and was assigned to Topic 1, which is probably the most reasonable.
3. Document 3 looks like an informal email discussing graphic programming and was reasonably assigned to Topic 1 which had a lot of words related to computers.
4. Document 4 looks like it is a programming algorithm. It's associations to the topics was relatively spread out; however, Topic 1 had strongest association.
5. Document 5 is a political document and was assigned to Topic 0, which is reasonable since Topic 0 is associated with opinions. 

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

1. 
* Decision Tree: like binary classification, points follow a tree path based on its value of a chosen feature for that node and its threshold at each split. Each leaf represents a different class that the point could be classified as. 
* KNN: classifies a points based on the classification of nearby points. Closer points will influence the classification of the chosen point more heavily.
* Random Forest: creates a collection of decision trees and performs decision tree classification on each. The classification with the highest vote is used as the predicted classification for the point on interest.
* Logistic Regression: performs multiclass classification by using the one vs rest scheme
* SVM RBF: operates similarly to KNN however only retains key "support" vectors for classification. Multi-class classification is most likely performed in the same manner as binary classification.

2. Transfer learning the use of pre-trained models that are fine tuned and applied to separate data of similar tasks. It is commonly used in NLP and computer vision  for quicker and better predictions/results.

3. This property is useful because you can extract the information at any transformation step based on the model/application you are constructing.


<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)